# Spotify App Development
Matt Morais<br>
DSC 672<br>
Software Presentation 1<br><br>
### Import Libraries
First import required libraries. For this demo, we will be using spotipy, a library in python that helps simplify interacting with the Spotify web API. We will see a lot of the functions line up with what the web API Reference states in the Spotify API documentation

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

### Create a spotify object
This object will be what we use to send requests to the server. This is a server based web API, so all the code will basically send commands to be executed on the server, and then we will receive a response back. The best part here, is will be when we send a query for a song or user, you will see that we do not need to know the data structure at all of the backend, we will just make a query and get a response with our results.<br> <br>
Aside from that, lets create our object using spotipy

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="9232804c01d545f785fcd7272b13b149",
                                                           client_secret=""))

Ok our object is created, lets run a quick query to see if it is functioning normally. Lets look for an artist and only get the first **5** songs from them that come up.

In [3]:
sp.search(q='artist:dance gavin dance', limit=5)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3Adance+gavin+dance&type=track&offset=0&limit=5',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6guC9FqvlVboSKTI77NG2k'},
       'href': 'https://api.spotify.com/v1/artists/6guC9FqvlVboSKTI77NG2k',
       'id': '6guC9FqvlVboSKTI77NG2k',
       'name': 'Dance Gavin Dance',
       'type': 'artist',
       'uri': 'spotify:artist:6guC9FqvlVboSKTI77NG2k'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'C

Now the response we get can be interpretted by using the WebAPI reference Spotify provides, lets look at that now and compare!<br> <br>
So how do we extract useful information from this response? We can look at it as a python dictionary / list of values! For instance, let's say we want to get just the track titles of these five objects that our query returned. We can do that by doing the following:

In [4]:
# save the response into a variable
results = sp.search(q='artist:dance gavin dance', limit=5)
# iterate over the response
for track in results['tracks']['items']:
    print(track['name'])

We Own The Night
Uneasy Hearts Weigh The Most
Feels Bad Man
Die Another Day
Cream Of The Crop


The `results['tracks']['items']` is simply breaking into the dictionary we got back as a response. So we are looking at the tracks key, and then the items key. Items, is a list of the track responses which is shown in the API refence as well<br><br>
We can also look up tracks that share the same name, and look at their artists using the same methodology - but with a slightly different query:

In [5]:
# save the response into a variable
results = sp.search(q='tracks:Someday', limit=5)
# iterate over the response
for track in results['tracks']['items']:
    print(track['album']['artists'][0]['name'])

Michael Bublé
City of the Sun
Misael Gauna
Peach Tree Rascals
SBI Audio Karaoke


### Create a CLI for a user to search for a song or artist
Refer to the web api reference for different query tags and what appropriate responses would be <br> **@ref Search For an Item > Query > qstring**<br> For this example, we will stick to either a track or an artists, since they output the same type of response.

In [6]:
storedSongList = []

def sortPop(key):
    return key['popularity']

while True:
    #query = input('What artist would you like to seach for: ')
    #if query == '':
        #break
    #results = sp.search(q='artist:'+query.lower().strip())

    results = sp.search(q='artist:dance gavin dance', limit = 50)
    # add artist, trackname, and popularity to dictionary of results
    trim_results = []
    for track in results['tracks']['items']:
        trim_results.append({'track': track['name'],
                             'artist': track['album']['artists'][0]['name'],
                             'popularity': track['popularity'],
                             'uri': track['uri']})
    # get top 5 most popular tracks
    top5 = []
    for k in trim_results:
        top5.sort(reverse=True,key=sortPop)
        if len(top5) == 5:
            i = 0
            for item in top5:
                if k['popularity'] > item['popularity']:
                    top5[i] = k
                    
                    break
                else:
                    i+=1
        else:
            top5.append(k)
    for item in top5:
        print(item['popularity'],' - ',item['track'])
        storedSongList.append(item['uri'])
    break

64  -  We Own The Night
60  -  Cream Of The Crop
60  -  Die Another Day
59  -  Uneasy Hearts Weigh The Most
58  -  For The Jeers


### Create a playlist on a user's account
To do this, the user **must have a premium account**. I paid for a premium account to demonstrate this functionality. First, we should verify that a user is premium. We adjust the auth manager to use SpotifyOAuth, so that we can check a user and interact with their account.<br>
<br>
see authorization scopes for definitions of the scope variable

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="9232804c01d545f785fcd7272b13b149",
                                               client_secret="",
                                               redirect_uri="http://localhost:1234/callback",
                                               scope="user-top-read user-library-read user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public playlist-modify-private playlist-read-collaborative playlist-read-private"))



Let's first look at the documentation from the spotify api web reference guide to see how we would look at a user profile, and what we can expect from it.<br> <br>

Now we can look at the user who is accessing our app to see if they are premium

In [14]:
me = sp.me()
me['product']

'premium'

This checks out, because my membership is indeed premium. If i were a non-premium user, this field would read **FREE**.

From spotipy reference guide:
    <br><code>user_playlist_create(user, name, public=True, collaborative=False, description='')</code>
    <br>Creates a playlist for a user
    <br>Parameters:
    <br>user - the id of the user
    <br>name - the name of the playlist
    <br>public - is the created playlist public
    <br>collaborative - is the created playlist collaborative
    <br>description - the description of the playlist`

So we first need to get my user ID from the me object:

In [16]:
userID = me['id']
print('User ID: ',userID)

User ID:  1297314535


In [17]:
newplaylist = sp.user_playlist_create(userID, 'MyNewPlaylist',public=True,collaborative=False,description='My example playlist for my Software Presentation')

Ok, now we have our new playlist. Let's add some songs to it using the top 5 tracks we found by dance gavin dance earlier!

In [20]:
playlistID = newplaylist['id']

In [21]:
results = sp.user_playlist_add_tracks(userID, playlistID, storedSongList)
print(results)

{'snapshot_id': 'MixhOGUwYzcwZGQ4YzcwYThkNzgyNTgwMGRhN2FkMzdiZGVhYTdjMzYw'}


### Play a playlist from a user's list of devices
Now that we made a playlist, let's play it from one of the user's registered devices. First, we need to get a list of the devices. It is typically easiest to look at the type, but we will look at the whole query response first.

In [22]:
devices = sp.devices()['devices']
devices

[{'id': '3e6d9bd9053b4e473e1a90cdb7b404f1f3c4edd0',
  'is_active': False,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'SM-F936U1',
  'type': 'Smartphone',
  'volume_percent': 100},
 {'id': 'f76096668319351c612435faf73efe05e5a01064',
  'is_active': False,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'DESKTOP-S7HPDK0',
  'type': 'Computer',
  'volume_percent': 100}]

From this, we can see i have two devices on my account. In reality, we could ask the user which device they would like to play the music on, for now, I will hard code in my desktop.

In [23]:
d_devices = {}
for device in devices:
    d_devices[device['type']] = device['id']
print(d_devices)

{'Smartphone': '3e6d9bd9053b4e473e1a90cdb7b404f1f3c4edd0', 'Computer': 'f76096668319351c612435faf73efe05e5a01064'}


Now let's get a list of all my playlists, and their IDs, so that we can select the one we want!

In [24]:
playlists = sp.current_user_playlists()['items']

playlistOptions = {}
for playlist in playlists:
    playlistOptions[playlist['name']]= playlist['uri']
    print(playlist['name'],playlist['uri'])

MyNewPlaylist spotify:playlist:7hZBlyAUlw4rYmQ0rHtrvM
ProjectExample spotify:playlist:464DX0Mov7pOrAdDYc7Nlw


And finally, start playback on my chosen device

In [3]:
sp.start_playback(d_devices['Computer'], playlistOptions['MyNewPlaylist'],None)

NameError: name 'd_devices' is not defined

# Update 1
Code for the first part of the update. Main part of the first update is getting the API working, the next part is beginning to acquire a data set. Let's first look at all songs from one artist, and see how we can search through different pages of query responses.

In [9]:
# let's say we want ten pages of responses (500 tracks)
pages = 10
tracksPerQuery = 50
queryTrackResponse = [] # store response into a list of tracks
for page in range(0,pages):
    print(page)
    results = sp.search(q='artist:dance gavin dance', \
                        limit = tracksPerQuery, \
                        offset = page*tracksPerQuery)
    items = results['tracks']['items']
    if len(items) == 0:
        print("Query responded with "+str(page)+" pages of results totalling "+str(len(queryTrackResponse))+" tracks")
        break
    else:
        for track in items:
            queryTrackResponse.append(track)

0
1
2
3
4
5
6
7
Query responded with 7 pages of results totalling 323 tracks


In [19]:
 for item in queryTrackResponse:
        print(item['popularity'],' - ',item['id'], ' - ', item['name'])

57  -  2DdluBZleLq30PlfUAqSD5  -  Inspire The Liars
58  -  0ZcQUwVyXgpUepJsvgOYgk  -  Death Of A Strawberry
50  -  5HllOy0J5LuH96EzMLGCPe  -  Head Hunter
51  -  4sM5Jb2eWDbizewXoVc1l0  -  Die Another Day
50  -  4MpXaXYhkGMw4gt3ZlS7sQ  -  One in a Million
52  -  2t2MD9sVs9PO1H3yphwWm2  -  Prisoner
53  -  46IUSULlL2SAmyyHHpXQJz  -  Lyrics Lie
51  -  4dRNNBipJ6wwSEEwa4FfcY  -  Strawberry's Wake
49  -  4PSMNdBxCQ02el8dYzjd7C  -  Nothing Shameful (feat. Andrew Wells)
50  -  639JHsrGV8A9qUYSXhOwsw  -  Summertime Gladness
54  -  6EGzKC0sQr7fAbCCumah8l  -  Betrayed By The Game
54  -  44Ssjb20kLO6FshLp0gNZE  -  Chucky vs. The Giant Tortoise
49  -  2BtNEbWuP3v4OQiEqcGQSf  -  Parody Catharsis
49  -  6egCRoFoyITQsNDmZPzq1q  -  Pop Off!
51  -  4NuFrArppAgXPZKCbImtiF  -  Deception
61  -  6G5JufGbj4GIVMG1ZVDCjW  -  Feels Bad Man
51  -  72Y5nO5FCZtq0w7T5JGbys  -  Young Robot
51  -  3QzhQ89DjOsDBjf79Fgpew  -  Man Of The Year
52  -  73XEi2vkcuHWF2m9wiOfbu  -  Care
51  -  06slOFQgdYZFNrTTf84UYR  -  Syner

### Extracting metadata from a track
Now i want to start gathering data for a data set. First, I want to see how we can get some useful data out of a track. I will start by querying a single song for its audio features <br> <br>
One of my favorite songs is Die Another Day, so let's look at that song first. Its audio ID can be found in the print statement above, but I will paste it here for reference.<br><br>
`51  -  4sM5Jb2eWDbizewXoVc1l0  -  Die Another Day`

In [25]:
heavy = sp.audio_features('4sM5Jb2eWDbizewXoVc1l0')
heavy

[{'danceability': 0.474,
  'energy': 0.983,
  'key': 6,
  'loudness': -3.332,
  'mode': 0,
  'speechiness': 0.118,
  'acousticness': 0.0675,
  'instrumentalness': 0,
  'liveness': 0.0825,
  'valence': 0.373,
  'tempo': 175.015,
  'type': 'audio_features',
  'id': '4sM5Jb2eWDbizewXoVc1l0',
  'uri': 'spotify:track:4sM5Jb2eWDbizewXoVc1l0',
  'track_href': 'https://api.spotify.com/v1/tracks/4sM5Jb2eWDbizewXoVc1l0',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4sM5Jb2eWDbizewXoVc1l0',
  'duration_ms': 221282,
  'time_signature': 3}]

As I would expect, this song has very high energy (which is ranked between 0 and 1). The others are also interesting, let's see how the numbers compare to an acoustic song by City and Colour

In [23]:
results = sp.search(q='artist:city and colour', \
                        limit = 10)

for item in results['tracks']['items']:
    print(item['name']+' - '+item['id'])

Comin' Home - 0SwUDplbBp66rM5sMC0eD7
The Grand Optimist - 58CHujf89H5cuuZqm4RaLn
We Found Each Other in the Dark - 4tVBfe1U2Q1n029LvKrc8o
Hello, I'm In Delaware - 1PUh1GENlvlgu0dW28I6tY
Lover Come Back - 7G9yE2L2bXxqaQKVL2rKAr
The Girl - 4ChDXpei5buHghjlcjkhul
Northern Wind - 65HOlqJpNqaUbateZw2GrA
Sleeping Sickness - 2SBFyml6gSXByXMRfW6UYi
The Girl - 1IFRVS4t1olI0XG9RBWdKH
As Much as I Ever Could - 2FKbisOVZcinq7VtYUFD4H


I know a particularly acoustic song is "as much as i ever could" so let's pick that one<br><br>
`As Much as I Ever Could - 2FKbisOVZcinq7VtYUFD4H`

In [27]:
acoustic = sp.audio_features('2FKbisOVZcinq7VtYUFD4H')
acoustic

[{'danceability': 0.56,
  'energy': 0.241,
  'key': 9,
  'loudness': -12.621,
  'mode': 0,
  'speechiness': 0.0341,
  'acousticness': 0.906,
  'instrumentalness': 0.000329,
  'liveness': 0.257,
  'valence': 0.148,
  'tempo': 118.462,
  'type': 'audio_features',
  'id': '2FKbisOVZcinq7VtYUFD4H',
  'uri': 'spotify:track:2FKbisOVZcinq7VtYUFD4H',
  'track_href': 'https://api.spotify.com/v1/tracks/2FKbisOVZcinq7VtYUFD4H',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2FKbisOVZcinq7VtYUFD4H',
  'duration_ms': 324787,
  'time_signature': 3}]

And here we see that acousticness is much higher, and energy is much lower - definitely what i would have expected. <br><br>
### Creating a function to turn a query response into a tab delimited file
Now I want to start gathering data that i can use as a training set. I think the easiest way of doing this is to store the data into a tab delimited file. For now, let's shoot for the following schema:<br>
`trackID, track name, artist, genre, audio features`

In [199]:
def query2tdf(results,writeHeader=False):
    '''Converts a query into a tab delimited file(tdf)'''
    with open('outfile.txt','a',encoding='utf-8') as oof:
        if writeHeader:
            oof.write('track\t')
            oof.write('trackID\t')
            oof.write('album\t')
            oof.write('artist\t')
            oof.write('genres\t')
            features = ['danceability','energy','key','loudness','mode', \
                        'speechiness','acousticness','instrumentals', \
                        'liveness','valence','tempo','type','id','uri','trackhref', \
                        'analysisuri','duration_ms','time_signature']
            for feature in features:
                oof.write(feature + '\t')
            oof.write('\n')
            
        items = results['tracks']['items']
        for item in items:
            track_name = item['name']
            track_id = item['id']
            artist = item['album']['artists'][0]['name']
            artist_id = item['album']['artists'][0]['id']
            album = item['album']['name']
            # get list of genres for artist
            genres = sp.artist(artist_id)['genres']
            # write to file
            oof.write(track_name + '\t')
            oof.write(track_id + '\t')
            oof.write(album + '\t')
            oof.write(artist + '\t')
            oof.write(','.join(genres) + '\t')
            # get audio features
            features = sp.audio_features(track_id)[0]
            for feature in features:
                oof.write(str(features[feature]) + '\t')
            oof.write('\n')

In [105]:
query2tdf(results)

### Query songs from a genre
Now let's see how we can query 100 pages from a given genre

In [107]:
result = sp.search(q='genre:Pop')

In [157]:
def limitedSearch(query,lim,offs):
    '''Performs a limited search for data specific to our project'''
    results = sp.search(q=query, limit=lim,offset=offs)
    
    track_Keepers = ['items']
    track_items_Keepers=['name','id','album']
    track_items_album_Keepers = ['artists','name','id']
    
    for k in results['tracks'].keys():
        if k not in track_Keepers:
            results['tracks'][k] = ''
        else:
            for songIdx in range(0,len(results['tracks']['items'])):
                
                for ke in results['tracks']['items'][songIdx].keys():
                    if ke not in track_items_Keepers:
                        results['tracks']['items'][songIdx][ke] = ''
                    else:
                        if ke == 'album':
                            for key in results['tracks']['items'][songIdx]['album']:
                                if key not in track_items_album_Keepers:
                                    results['tracks']['items'][songIdx]['album'][key] = ''
                                
                
    return results

In [197]:
result = limitedSearch('genre:Pop',50,0)
query2tdf(result,True)

Wrote 0 tracks
Wrote 5 tracks
Wrote 10 tracks
Wrote 15 tracks
Wrote 20 tracks
Wrote 25 tracks
Wrote 30 tracks
Wrote 35 tracks
Wrote 40 tracks
Wrote 45 tracks


In [ ]:
import time

#list of genres
genres = sp.recommendation_genre_seeds()['genres']

numPages = 19
output = ''
for year in range(2012,2022,1):
    print('Year: '+str(year))
    for genre in genres:
        print('  Genre: '+genre)

        for pageIdx in range(0,numPages):
            result = limitedSearch('genre:'+genre+',year:'+str(year),50,pageIdx*50)
            if output == '':
                output = result
            else:
                for track in result['tracks']['items']:
                    output['tracks']['items'].append(track)
        query2tdf(output)

Year: 2012
  Genre: acoustic
  Genre: afrobeat
  Genre: alt-rock
  Genre: alternative
  Genre: ambient
  Genre: anime
  Genre: black-metal
  Genre: bluegrass
  Genre: blues
  Genre: bossanova
  Genre: brazil
  Genre: breakbeat
  Genre: british
  Genre: cantopop
  Genre: chicago-house
  Genre: children
  Genre: chill
  Genre: classical
  Genre: club
  Genre: comedy
  Genre: country
  Genre: dance
  Genre: dancehall
  Genre: death-metal
  Genre: deep-house
  Genre: detroit-techno
  Genre: disco
  Genre: disney
  Genre: drum-and-bass
  Genre: dub
  Genre: dubstep
  Genre: edm
  Genre: electro
  Genre: electronic
  Genre: emo
  Genre: folk
  Genre: forro
  Genre: french
  Genre: funk
  Genre: garage
  Genre: german
  Genre: gospel
  Genre: goth
  Genre: grindcore
  Genre: groove
  Genre: grunge
  Genre: guitar
  Genre: happy
  Genre: hard-rock
  Genre: hardcore
  Genre: hardstyle
  Genre: heavy-metal
  Genre: hip-hop
  Genre: holidays
  Genre: honky-tonk
  Genre: house
  Genre: idm
  Genre

  Genre: r-n-b
  Genre: rainy-day
  Genre: reggae
  Genre: reggaeton
  Genre: road-trip
  Genre: rock
  Genre: rock-n-roll
  Genre: rockabilly
  Genre: romance
  Genre: sad
  Genre: salsa
  Genre: samba
  Genre: sertanejo
  Genre: show-tunes
  Genre: singer-songwriter
  Genre: ska
  Genre: sleep
  Genre: songwriter
  Genre: soul
  Genre: soundtracks
  Genre: spanish
  Genre: study
  Genre: summer
  Genre: swedish
  Genre: synth-pop
  Genre: tango
  Genre: techno
  Genre: trance
  Genre: trip-hop
  Genre: turkish
  Genre: work-out
  Genre: world-music
Year: 2016
  Genre: acoustic
  Genre: afrobeat
  Genre: alt-rock
  Genre: alternative
  Genre: ambient
  Genre: anime
  Genre: black-metal
  Genre: bluegrass
  Genre: blues
  Genre: bossanova
  Genre: brazil
  Genre: breakbeat
  Genre: british
  Genre: cantopop
  Genre: chicago-house
  Genre: children
  Genre: chill
  Genre: classical
  Genre: club
  Genre: comedy
  Genre: country
  Genre: dance
  Genre: dancehall
  Genre: death-metal
  Ge

  Genre: iranian
  Genre: j-dance
  Genre: j-idol
  Genre: j-pop
  Genre: j-rock
  Genre: jazz
  Genre: k-pop
  Genre: kids
  Genre: latin
  Genre: latino
  Genre: malay
  Genre: mandopop
  Genre: metal
  Genre: metal-misc
  Genre: metalcore
  Genre: minimal-techno
  Genre: movies
  Genre: mpb
  Genre: new-age
  Genre: new-release
  Genre: opera
  Genre: pagode
  Genre: party
  Genre: philippines-opm
  Genre: piano
  Genre: pop
  Genre: pop-film
  Genre: post-dubstep
  Genre: power-pop
  Genre: progressive-house
  Genre: psych-rock
  Genre: punk
  Genre: punk-rock
  Genre: r-n-b
  Genre: rainy-day
  Genre: reggae
  Genre: reggaeton
  Genre: road-trip
  Genre: rock
  Genre: rock-n-roll
  Genre: rockabilly
  Genre: romance
  Genre: sad
  Genre: salsa
  Genre: samba
  Genre: sertanejo
  Genre: show-tunes
  Genre: singer-songwriter
  Genre: ska
  Genre: sleep
  Genre: songwriter
  Genre: soul
  Genre: soundtracks
  Genre: spanish
  Genre: study
  Genre: summer
  Genre: swedish
  Genre: syn

In [180]:
query2tdf(output)

File Written


In [185]:
sp.recommendation_genre_seeds()['genres']

['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',